In [13]:
import pandas as pd
import webbrowser

js = "window.scrollTo(0, document.body.scrollHeight);"

In [7]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-BAUvNUjp2AeV_daeeqHReX0M3ew3ZpEL3nfkrz96uUd816mV_hV1uWMvbsACphEBGjqHJBswGwFz/pub?gid=614465369&single=true&output=csv")

In [8]:
# filter dataframe depending on the value of two columns
df = df[(df['Curso Escolar'] == "2022-23") & (df['Convocatoria'] == 1)]

In [22]:
# open the first row in a new tab
def open_url(row):
    course_id = row["Moodle_url"].split("id=")[-1]
    url = f"https://moodle.catedu.es/course/completion.php?id={course_id}#id_rolescontainer"
    webbrowser.open_new_tab(url)

# apply the function to the dataframe
df[2:10].apply(open_url, axis=1)

2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [18]:
df["Tutorizado"]

0      False
1      False
2      False
3      False
4      False
       ...  
171    False
172    False
173    False
174    False
175    False
Name: Tutorizado, Length: 140, dtype: object